In [ ]:
def online_monte_carlo_policy_improvement(env, policy, num_episodes, gamma=1.0, alpha=0.1, epsilon=0.1):
    # Initialize value function
    V = np.zeros(env.nS)

    for i_episode in range(num_episodes):
        # Generate an episode
        episode = []
        state = env.reset()
        done = False
        while not done:
            # Generate episode. Choose action based on epsilon-greedy policy. 
            probs = np.ones(env.nA) * epsilon / env.nA
            probs[np.argmax(policy[state])] += (1.0 - epsilon)
            action = np.random.choice(env.nA, p=probs)
            next_state, reward, done, _ = env.step(action)
            episode.append((state, action, reward))
            state = next_state

        # Update the value function
        G = 0
        for t in range(len(episode) - 1, -1, -1):
            state, action, reward = episode[t]
            next_state = episode[t+1][0] if t < len(episode) - 1 else None
            G = gamma * G + reward
            V[state] += alpha * (G - V[state])
            if next_state is not None and action != np.argmax(policy[next_state]):
                # Update policy based on new value function estimate
                Q = np.zeros(env.nA)
                for a in range(env.nA):
                    Q[a] = sum([p * (r + gamma * V[s_]) for p, s_, r, _ in env.P[state][a]])
                policy[state] = np.eye(env.nA)[np.argmax(Q)]

    return policy